In [13]:
import torch
import torch.nn as nn
import numpy as np
import json


class LIMUBERTModel(nn.Module):
    def __init__(self):
        super(LIMUBERTModel, self).__init__()
        self.embed = nn.Linear(6, 72)  # Adjusted to match the saved model
        self.transformer = nn.Transformer(d_model=72, nhead=8, num_encoder_layers=6)  # Adjusted to match the saved model
        self.fc = nn.Linear(72, 72)  # Adjusted to match the saved model

    def forward(self, x):
        x = self.embed(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

    def get_embeddings(self, x):
        x = self.embed(x)
        return x


---------------------


In [14]:

# Instantiate the model
model = LIMUBERTModel()

# Path to your model file
model_path = 'saved/pretrain_base_hhar_20_120/hhar.pt'

# Load the state dictionary
state_dict = torch.load(model_path)

# Load the state dict into the model
model.load_state_dict(state_dict, strict=False)
state_dict = torch.load(model_path, map_location=torch.device('cpu'))

# Set model to evaluation mode
model.eval()

LIMUBERTModel(
  (embed): Linear(in_features=6, out_features=72, bias=True)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=72, out_features=72, bias=True)
          )
          (linear1): Linear(in_features=72, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=72, bias=True)
          (norm1): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): _LinearWithBias(in_features=72, out_features=72, bias=True)
          )
          (linear1):